# <font color='darkgreen'>DATA SCIENCE EM MARKETING</font>
## <font color='darkgreen'>SEGMENTAÇÃO DE CLIENTES</font>
## <font color='darkgreen'>Machine Learning - Estatística - Análise e Visualização de Dados</font>

Este Projeto envolve o uso de aprendizado não supervisionado para a segmentação de clientes, com o objetivo de encontrar grupos similares e ajudar a equipe de Marketing a compreender como orientar melhor as campanhas, de acordo com o perfil dos clientes. 

## Instalar e Carregar Pacotes

In [ ]:
# Usado para resolver bug do K-Means
!pip install -q threadpoolctl==3.5.0

In [ ]:
# Importar pacotes

# Manipulação de dados
import numpy as np
import pandas as pd

# Estatística e Machine Learning
from scipy import stats
import threadpoolctl
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Gravar versões de pacotes
!pip install -q -U watermark

In [ ]:
# Versões dos pacotes
%reload_ext watermark
%watermark --iversions

## Carregar Dados

In [ ]:
# Garantir reprodutibilidade
np.random.seed(42)

In [ ]:
# Carregar dataset
df = pd.read_csv('dataset.csv')

In [ ]:
# Shape
df.shape

In [ ]:
# Info
df.info()

In [ ]:
# Dados iniciais
df.head()

In [ ]:
# Dados aleatórios
df.sample(10)

## Análise Exploratória e Limpeza de Dados

In [ ]:
# Verificar valores nulos
print(df.isnull().sum())

O algoritmo KMeans pode ser impactado por valores outliers e pela correlação entre as variáveis, pois utiliza a distância euclidiana entre os pontos para formar clusters e o cálcuo de distância, vamos usar boxplot e uma matriz de correlação para analisar os dados.

In [ ]:
# Plotar boxplots
df.plot(subplots=True, layout=(3,3), kind='box', figsize=(12,14), patch_artist=True,
            boxprops=dict(facecolor='#006400'))

# Ajustar espaçamento
plt.subplots_adjust(wspace=0.1)

In [ ]:
# Observe que a variável cancelou é categórica
df.Cancelou.value_counts()

### Identificar os outliers

In [ ]:
# Definir o Intervalo Interquartil
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
# Verificar os valores que estão acima ou abaixo do IQR
outliers = ((df < (Q1 - 2.5 * IQR)) | (df > (Q3 + 2.5 * IQR))).any(axis = 1)

In [ ]:
# Filtrar os outliers
df_outliers = df[outliers]

In [ ]:
# Visualizar os outliers
df_outliers

Identificados os outliers vamos, tratar deles logo à frente!

In [ ]:
# Calcular a matriz de correlação
correlation_matrix = df.corr()

In [ ]:
correlation_matrix

In [ ]:
# Plot
plt.figure(figsize = (10,8))
sns.heatmap(correlation_matrix, annot = True, cmap = 'YlGnBu')
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
# Criar o padronizador
scaler = StandardScaler()

In [ ]:
# Padronizando as Variáveis
df_pd = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

## Definir o Valor de K em Modelos de Clusterização


### Método Elbow

O Método do Cotovelo é uma técnica usada para determinar o número ideal de grupos em uma análise de cluster, como o K-means.
O gráfico resultante ajuda a identificar o ponto onde a redução na inércia começa a diminuir significativamente. Este ponto é considerado o número ideal de clusters para o de dados.

In [ ]:
# Lista para armazenar o SSE
sse = []

In [ ]:
# Range de valores de k a serem testados
k_range = range(1, 11)

In [ ]:
# Usando o Kmeans para testar 10 modelos
for k in k_range:
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(df_pd)
    sse.append(kmeans.inertia_)

In [ ]:
# Plot
plt.plot(k_range, sse, 'bx-')
plt.xlabel('Número de Clusters')
plt.ylabel('Soma dos Quadrados dentro do Cluster')
plt.title('Método Elbow')
plt.show()

O gráfico do método Elbow sugere um ponto de inflexão em torno de 3 a 4 clusters.

### Método da Silhueta 

O Método da Silhueta é outra técnica usada para determinar o número ideal de clusters em uma análise de cluster. Ele avalia a qualidade dos clusters medindo quão semelhantes os pontos dentro de um cluster são uns aos outros (coesão) em comparação com os pontos de outros clusters (separação). A pontuação da silhueta varia de -1 a 1, onde valores próximos a 1 indicam que os pontos estão bem agrupados, valores próximos a 0 indicam que os pontos estão na fronteira entre dois clusters, e valores negativos indicam que os pontos podem ter sido atribuídos ao cluster errado.

In [ ]:
# Lista para o Silhouette score
sil_score = []

# Loop pelo range de valores de k a serem testados
for k in k_range:
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(df_pd)
    
    # Silhouette score não é definido para k = 1, então filtramos
    if k != 1:
        sil_score.append(silhouette_score(df_pd, kmeans.labels_))

# Plot
plt.plot(k_range[1:], sil_score, 'bx-')
plt.xlabel('Número de Clusters')
plt.ylabel('Coeficiente de Silhueta')
plt.title('Método Silhueta')
plt.show()

## Construir a Primeira Versão do Modelo de Clusterização

In [ ]:
# Modelo com 4 clusters
modelo_kmeans = KMeans(n_clusters = 4, random_state = 42)

In [ ]:
# Treinar o modelo e salvar o resultado em uma coluna do df original
df['cluster'] = modelo_kmeans.fit_predict(df_pd)

In [ ]:
# Amostra aleatória de dados
df.sample(10)

In [ ]:
df.shape

In [ ]:
# Usar o dataset original, incluindo os outliers
df_limpo = df

In [ ]:
df_limpo.shape

### PairGrid

In [ ]:
# Criar um mapa de cores
palette = sns.color_palette('dark', n_colors = len(df_limpo['cluster'].unique()))
color_map = dict(zip(df_limpo['cluster'].unique(), palette))

# Plot
g = sns.PairGrid(df_limpo, hue = 'cluster', palette = color_map, diag_sharey = False)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw = 2)
plt.show()

# Visualizar o mapa de cores
for cluster, color in color_map.items():
    plt.scatter([], [], c = [color], label = f'Cluster {cluster}')
plt.legend(title = 'Legenda de Cores dos Clusters')
plt.axis('off')
plt.show()

## Construir a Segunda Versão do Modelo de Clusterização

In [ ]:
# Modelo com 3 clusters
modelo_kmeans = KMeans(n_clusters = 3, random_state = 42)

In [ ]:
df.drop('cluster', axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
# Treinar o modelo e salvar o resultado em uma coluna do dataframe original
df['cluster'] = modelo_kmeans.fit_predict(df_pd)

In [ ]:
# Amostra aleatória de dados
df.sample(10)

In [ ]:
# Mantém apenas as linhas que não são outliers
df_limpo = df[~outliers]

In [ ]:
# Criar um mapa de cores
palette = sns.color_palette('dark', n_colors = len(df_limpo['cluster'].unique()))
color_map = dict(zip(df_limpo['cluster'].unique(), palette))

# Plot
g = sns.PairGrid(df_limpo, hue = 'cluster', palette = color_map, diag_sharey = False)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw = 2)
plt.show()

# Visualizar o mapa de cores
for cluster, color in color_map.items():
    plt.scatter([], [], c = [color], label = f'Cluster {cluster}')
plt.legend(title = 'Legenda de Cores dos Clusters')
plt.axis('off')
plt.show()

## Análise dos Clusters

In [ ]:
df_limpo[df_limpo.cluster == 0].head()

In [ ]:
df_limpo[df_limpo.cluster == 0].mean()

In [ ]:
df_limpo[df_limpo.cluster == 1].head()

In [ ]:
df_limpo[df_limpo.cluster == 1].mean()

In [ ]:
df_limpo[df_limpo.cluster == 2].head()

In [ ]:
df_limpo[df_limpo.cluster == 2].mean()

### Centróides 

Os centroides são os pontos médios dos clusters que representam as "médias" ou "centros" dos dados agrupados em cada cluster.Eles fornecem uma maneira de resumir as características principais de cada cluster e são úteis para entender a distribuição e a variação dos dados dentro dos clusters.

In [ ]:
# Extraindo os centróides
centroides = modelo_kmeans.cluster_centers_

In [ ]:
centroides

Cada linha representa um cluster com as seis variáveis (colunas).

### Plot dos Clusters

In [ ]:
# Cria a figura
plt.figure(figsize = (8, 6))

# Loop pelos clusters
for cluster_num in range(3):
    mask = df['cluster'] == cluster_num
    plt.scatter(df_pd[mask].iloc[:, 0], df_pd[mask].iloc[:, 1], label = f'Cluster {cluster_num}')

# Plot
plt.scatter(modelo_kmeans.cluster_centers_[:, 0], 
            modelo_kmeans.cluster_centers_[:, 1], 
            s = 300, 
            c = 'blue', 
            marker = 'X', 
            label = 'Centróides')
plt.legend()
plt.title("Plot de Agrupamento")
plt.show()

**Interpretação**:


**Cluster 0**: Média de Idade: 37 anos; Gasto Mensal: 213 reais; Tempo de Assinatura: 15 meses; Taxa de Uso: 0.63; 
Suporte Tickets: 1.6 e Cancelamento: 51%.

O Cluster 0 (primeira linha dos centróides) contém em sua maioria clientes de ambos os grupos, cancelou e não cancelou. Observe que os valores estão próximos de 0, o que sugere que este cluster pode estar centrado próximo à média do conjunto de dados. Esse é o grupo intermediário.


**Cluster 1**: Média de Idade: 55 anos; Gasto Mensal: 277 reais; Tempo de Assinatura: 20 meses; Taxa de Uso: 0.90;
Suporte Tickets: 2.16 e Cancelamento: 89%

O Cluster 1 (segunda linha) contém em sua maioria clientes do grupo que cancelou a assinatura. Os valores são positivos e superiores a 1 para a maioria das dimensões, o que sugere que as observações neste cluster têm valores superiores à média para estas dimensões.


**Cluster 2**: Média de Idade: 26 anos; Gasto Mensal: 99 reais; Tempo de Assinatura: 6 meses; Taxa de Uso: 0.29
Suporte Tickets: 0.77 e Cancelamento: 9%

O Cluster 2 (terceira linha) contém em sua maioria clientes do grupo que não cancelou a assinatura. Os valores são negativos e inferiores a -1 para a maioria das dimensões, o que sugere que as observações neste cluster têm valores inferiores à média para estas dimensões.

Veja quantas observações foram atribuídas a cada cluster para entender o tamanho deles.

In [ ]:
df['cluster'].value_counts()

## Métrica da Qualidade dos Clusters

### Análise de Silhueta

A Análise do Método Silhueta é uma técnica utilizada para avaliar a qualidade do agrupamento em análises de cluster. Ela ajuda a determinar o quão bem cada ponto foi agrupado e quão apropriado é o número de clusters escolhido

Interpretação do Método Silhueta
Coeficiente de Silhueta (Silhouette Score):

Cada ponto de dados tem um valor de silhueta que varia de -1 a 1.
Um valor de silhueta próximo a 1 indica que o ponto está bem dentro de seu próprio cluster e longe de outros clusters.
Um valor próximo de 0 indica que o ponto está na fronteira ou sobreposição de dois clusters.
Um valor negativo indica que o ponto pode ter sido agrupado no cluster errado.

In [ ]:
df.groupby('cluster').mean()

In [ ]:
score = silhouette_score(df_pd, df['cluster'])
print(score)